In [1]:
!conda install -c anaconda beautifulsoup4

done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.0       |           py37_0         147 KB  anaconda
    ca-certificates-2019.8.28  |                0         133 KB  anaconda
    certifi-2019.9.11          |           py37_0         154 KB  anaconda
    conda-4.8.2                |           py37_0         3.0 MB  anaconda
    openssl-1.1.1d             |       h1de35cc_2         3.4 MB  anaconda
    ------------------------------------------------------------
                                           Total:         6.9 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  beautifulsoup4                                  pkgs/main --> anaconda
  ca-certificates                                 pkgs/main --> anaconda
  certifi       

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
postal_code_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(postal_code_data, 'html.parser')
postal_list=[]
borough_list=[]
neighbor_list=[]
table = soup.find('table')
table_rows = table.find_all('tr')

for tr in table_rows:
    td = tr.find_all('td')
    if (len(td)> 0):
        postal_list.append(td[0].text)
        borough_list.append(td[1].text)
        neighbor_list.append(td[2].text.rstrip('\n'))
        

In [4]:
df = pd.DataFrame({"Postal Code": postal_list, "Borough": borough_list, "Neighborhood": neighbor_list})
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df_without_not_assgined = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_without_not_assgined.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [8]:
df_grouped_by_borough = df_without_not_assgined.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped_by_borough.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
# if neighborhood is not assigned we should assign the boroughs value to the neighborhood
for index, row in df_grouped_by_borough.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

df_grouped_by_borough.head(12)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [13]:
df_grouped_by_borough.shape

(103, 3)